Define your variables (That's the only cells you need to change). Then run the whole notebook

In [1]:
image_path = '/home/tli_project/Desktop/Data/T4/stage1/211025/4D_GFP_GFP_SyNRA_4D_Rigid_211025_P36.5.tif'
save_path = '/home/tli_project/Desktop/Data/T4/stage1/211025/'
save_name = '211025_P36.5'

#N2V parameters
model_name ='n2v_3D_v6'
model_path ='/home/tli_project/Desktop/TLI_preprocessing/Work_folder/N2V/models/'

#Clahe parameters:
clipLimit = 1
kernel_size = (45, 45)

In [2]:
if save_path[-1] != '/':
    save_path += '/'

Load needed packages

In [3]:
import tifffile as tif
import numpy as np
import utils.denoise as denoise
import utils.datautils as datautils
import Neurosetta as neu
import matplotlib.pyplot as plt

2022-06-23 20:42:42.773342: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
# Read 4D GFP image
image_4D = tif.imread(image_path)

In [5]:
# image_4D -= 30000
# save_file = save_path+save_name+'_masked.tif'
# tif.imwrite(save_file, image_4D, imagej=True, resolution=(1./0.076, 1./0.076),
#                 metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

In [6]:
image_4D = denoise.N2V_4D(image_4D, model_name=model_name, model_path=model_path, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name) 

2022-06-23 20:42:47.716025: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-23 20:42:47.716171: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tli_project/miniconda3/envs/preprocessing/lib/python3.8/site-packages/cv2/../../lib64:/app/lib
2022-06-23 20:42:47.716178: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-23 20:42:47.716193: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: l-p0-345
2022-06-23 20:42:47.716195: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: l-p0-345
2022-06-23 20:42:47.716262: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find lib

Loading network weights from 'weights_best.h5'.


applying N2V:   0%|          | 0/86 [00:00<?, ?it/s]

The input image is of type int16 and will be casted to float32 for prediction.


2022-06-23 20:42:48.818849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-23 20:42:48.836905: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2496000000 Hz
2022-06-23 20:42:49.010420: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 742137856 exceeds 10% of free system memory.
2022-06-23 20:42:49.339794: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 742137856 exceeds 10% of free system memory.
2022-06-23 20:42:51.562357: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 742137856 exceeds 10% of free system memory.
2022-06-23 20:42:51.600875: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1484275712 exceeds 10% of free system memory.
2022-06-23 20:42:51.722174: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 742137856 exceeds 10% of free system memory.
applying N2V:   1%|   

...

applying N2V:  98%|█████████▊| 84/86 [4:23:05<06:15, 187.62s/it]

The input image is of type int16 and will be casted to float32 for prediction.


applying N2V:  99%|█████████▉| 85/86 [4:26:12<03:07, 187.64s/it]

The input image is of type int16 and will be casted to float32 for prediction.


applying N2V: 100%|██████████| 86/86 [4:29:20<00:00, 187.91s/it]
/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning: <tifffile.TiffWriter 'N2V_211025_P36.5.tif'> truncating ImageJ file
  warnings.warn(


In [18]:
# remove further noise pixels before applying clahe, by removing 95% quantil
# threshold = np.quantile(image_4D,0.95)
# # threshold = 4 * image_4D.mean()
# threshold = 300
# image_4D[image_4D<threshold] = 0
# image_4D[mask == 0] = 0

In [5]:
# image_4D.min(), image_4D.max()

(7374, 39496)

In [7]:
# image_4D = image_4D[:,3:31]
image_4D = denoise.clahe_4D(image_4D, kernel_size=kernel_size, clipLimit=clipLimit, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name)

applying clahe: 100%|██████████| 86/86 [03:58<00:00,  2.77s/it]
/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning: <tifffile.TiffWriter 'clahe_211025_P36.5.tif'> truncating ImageJ file
  warnings.warn(


In [ ]:
# tif.imwrite(save_path+save_name+'1', image_4D, bigtiff=True, resolution=(1./1, 1./1),
#                 metadata={'spacing': 1, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

In [ ]:
mask = denoise.mask_4D(image_4D, return_mask=False, 
                        xy_pixel=0.076, z_pixel=0.4, 
                        sig=1, save=True, 
                        save_path=save_path, 
                        save_file=save_name)

masking the image: 100%|██████████| 93/93 [03:20<00:00,  2.15s/it]
/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning: <tifffile.TiffWriter 'mask_SyNRA_4D_211114_P36.tif'> truncating ImageJ file
  warnings.warn(


In [14]:
mask.min(), mask.max(), mask.mean(), np.quantile(mask, 0.999)

(0, 54287, 117.13803814904556, 28049.0)

In [8]:
mask[mask>0] = 1
image_4D = image_4D * mask
save_file = save_path+save_name+'_masked.tif'
tif.imwrite(save_file, image_4D, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning: <tifffile.TiffWriter 'SyNRA_4D_211114_P36_masked.tif'> truncating ImageJ file
  warnings.warn(


In [15]:
# # masking the image using neurosetta
# image_4D = neu.Neuron(image_4D)
# mask =  neu.mask_Neuron(image_4D)
# image_4D = image_4D.array
# mask = mask.array
# for i, img in enumerate(image_4D):
#     image_4D[i] = datautils.img_limits(mask[i], limit=img.max())
# image_4D = image_4D.astype('int16')
# datautils.save_image(save_path+'mask2_'+save_name, image_4D)

In [7]:
print('memory cleared, and kernel stopped')
print('Job is DONE!')
os._exit(00)

Error: Canceled future for execute_request message before replies were done